# SUMO to TrajDataFrame

This repository contains scripts and methods to convert a [FCD](https://sumo.dlr.de/docs/Simulation/Output/FCDOutput.html) (Floating Car Data) simulation output from SUMO to a `TrajDataFrame` able to be used in [scikit-mobility](https://github.com/scikit-mobility/scikit-mobility) package. This sample notebook shows how to handle SUMO FCD data out to `scikit-mobility` and plot vehicle trajectories in a Folium map. Data comes from a random vehicle flow simulation of the city centre of [Treviso](https://en.wikipedia.org/wiki/Treviso) (my hometown).

In [1]:
# Import packages
import skmob
import pandas as pd
import sys
import os

# SUMO Simulation
This section runs a SUMO simulation over random flows taken from OSM with the built-in importer (see [this tutorial](https://sumo.dlr.de/docs/Tutorials/OSMWebWizard.html)). You can skip this section if you already have the output FCD .xml file.

In [2]:
# Run Simulation and get output file
os.system("sumo -c data/osm.sumocfg \
            --fcd-output data/fcd_out.xml \
            --device.fcd.probability 1 \
            --fcd-output.geo")

# Convert output from xml to csv (check xml2csv.py location in your SUMO installation)
os.system("python3 /usr/share/sumo/tools/xml/xml2csv.py data/fcd_out.xml")

512

# DataFrame and TrajDataFrame
Here we take the .csv Floating Car Data file and convert it to `DataFrame` and `TrajDataFrame`. Since the SUMO output files is in seconds relative to the simulation period, we add them to the current `datetime` to build a full `datetime` column.

In [3]:
# Read .csv into a Pandas DataFrame
df = pd.read_csv('data/fcd_out.csv', sep=';')

# Convert datetime column - steps to datetime - current datetime + delta
df['datetime'] = pd.to_datetime('now', utc=True) + pd.to_timedelta(df['timestep_time'], unit='s')

# TrajDataFrame from Dataframe
tdf = skmob.TrajDataFrame(df, 
                          latitude='vehicle_y', 
                          longitude='vehicle_x', 
                          datetime='datetime', 
                          user_id='vehicle_id')

# Plot Vehicle Trajectories
You can now plot the vehicle trajectories over a Folium interactive map, showing start and end position of each trip.

In [4]:
tdf.plot_trajectory(zoom=14, weight=3, opacity=0.9, tiles='Stamen Toner')